In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity

import torch

c:\Users\SWASTIK\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%%time

from nltk.corpus import stopwords 
from nltk import download 
from pyemd import emd
import gensim
from gensim.models import Word2Vec

download('stopwords')
stop_words = stopwords.words('english')

CPU times: total: 3.89 s
Wall time: 6.47 s


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SWASTIK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
model = gensim.models.KeyedVectors.load_word2vec_format("D:\Aiproject\Automatic-Answer-checker-\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin", binary=True)  

In [5]:
model.init_sims(replace=True)

C:\Users\SWASTIK\AppData\Local\Temp\ipykernel_17424\1992334471.py:1: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [8]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [9]:
def cos_sim(input_vectors):
    similarity = cosine_similarity(input_vectors)
    return similarity
negative = ["not" , "without","against","bad","useless","no","dislike","hate"]

def semantic_similarity(actual_answer , given_answer) :
    actual = actual_answer.lower().split(".")
    given = given_answer.lower().split(".")
    
    
    
    sim_checker = actual 
    
    not_matching_semantics = list()
    
    semantic_1 = 0   # Actual_answer
    semantic_2 = 0   # Given_answee
    
    actual_embed_list = list()
    given_embed_list = list()
    
    
    
    for z in range(len(actual)) :
        list_actual = list()  
        list_actual.append(actual[z])
        actual_embed_list.append(embed(list_actual))
        #print(actual_embed_list[z].shape)
    
    for z in range(len(given)) :
        
        semantic_1 = 0
        semantic_2 = 0 
        list_given = list()
        list_given.append(given[z])
        embed_z = embed(list_given)
        
        
        sim_check = sim_checker.copy() 
        sim_check.append(given[z]) 
        
        sen_em = embed(sim_check)
        
        similarity_matrix = cos_sim(np.array(sen_em))
        
        similarity_matrix_df = pd.DataFrame(similarity_matrix) 
        
        cos_list = list(similarity_matrix_df[len(similarity_matrix_df) - 1]) 
        cos_list = cos_list[:len(cos_list)-1]
        #print(cos_list)
        
        index = cos_list.index(max(cos_list))
        
        actual_check = actual[index]
        actual_check = actual_check.split()
        for i in range(len(actual_check) - 1) :
            if(actual_check[i] in negative and actual_check[i+1] in negative) :
                semantic_1 += 1 
            elif(actual_check[i] in negative and actual_check[i+1] not in negative) :
                semantic_1 -= 1 
                
        
        


        answer_given = given[z].split()
        for i in range(len(answer_given) - 1) :
            if(answer_given[i] in negative and answer_given[i+1] in negative) :
                semantic_2 += 1 
            elif(answer_given[i] in negative and answer_given[i+1] not in negative) :
                semantic_2 -= 1 

                
        
        if(semantic_1 == 0 and semantic_2 == 0) :
            
            """
            Well and good
            """
        elif(semantic_1 < 0  and semantic_2 >= 0) :
            not_matching_semantics.append(list([actual[index],given[z]]))
            embed_z*=(-1)
            
        
        elif(semantic_1 >= 0 and semantic_2 < 0 ) :
            not_matching_semantics.append(list([actual[index],given[z]]))
            embed_z*=(-1)
            
        
        
        #print(semantic_1,semantic_2,actual[index],given[z])
        
        
        given_embed_list.append(embed_z)
        
        
        
    
    #print(np.array(actual_embed_list).shape)
    actual_embed = actual_embed_list[0] 
    #print(actual_embed.shape) 
    
    for i in range(len(actual_embed_list)-1) :
        #print(actual_embed_list[i+1].shape)
        actual_embed += actual_embed_list[i+1]
        
    given_embed = given_embed_list[0] 
    for i in range(len(given_embed_list) - 1) :
        given_embed += given_embed_list[i+1] 
        
        
    
            
    actual_embed = np.array(actual_embed).reshape(512)
    given_embed = np.array(given_embed).reshape(512) 
    sem_checker = list([actual_embed,given_embed]) 
    answer = pd.DataFrame(cos_sim(sem_checker))
            
        
    return not_matching_semantics , answer[0][1]